<a href="https://colab.research.google.com/github/zbutton314/CS-5560/blob/main/Lab4/code/ICP_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

### System Installations

In [1]:
# Install Java
!sudo apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install Spark using Hadoop
!wget -q https://downloads.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz

# Unzip the downloaded folder
!tar xf spark-3.0.3-bin-hadoop3.2.tgz

# findspark will find the Spark installation on this machine
!pip install -q findspark
!pip install pyspark==3.0.2

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/u

### Spark Installation

In [2]:
import os
import findspark
from pyspark.sql import SparkSession

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] =  "/content/spark-3.0.3-bin-hadoop3.2"

findspark.init()
print(f"Spark location: {findspark.find()}")

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

spark

Spark location: /content/spark-3.0.3-bin-hadoop3.2


### Google Drive Setup

In [3]:
from google.colab import drive
drive.mount('/content/drive')

icp4_data_path = "/content/drive/MyDrive/data/ICP-4"

Mounted at /content/drive


# Import Data

In [4]:
df = spark.read.csv(f"{icp4_data_path}/data.csv", header=True, inferSchema= True)
df.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: string (nullable = true)
 |-- Churn: string (nullable = true)



In [17]:
# Function to display information about Spark dataframe
def disp(spark_df, n=5):
  print(f"Rows: {spark_df.count()}")
  print(f"Columns: {len(spark_df.columns)}")
  spark_df.show(n)

# Try three transformations

In [22]:
# 1) Filter
df_female = df.filter(df.gender == "Female")
disp(df_female, 10)

Rows: 3488
Columns: 21
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|7590-VHVEG|Female|            0|    Y

In [19]:
# 2) Sample
df_random_half = df.sample(withReplacement=False, fraction=0.5, seed=314)
disp(df_random_half, 10)

Rows: 3505
Columns: 21
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|7590-VHVEG|Female|            0|    Y

In [21]:
# 3) Union
df_new = df_female.union(df_random_half)
disp(df_new, 10)

Rows: 6993
Columns: 21
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|7590-VHVEG|Female|            0|    Y

# Try three actions

In [26]:
# 1) Take Sample
print(df.rdd.takeSample(withReplacement=True, num=10, seed=1618))

[Row(customerID='2495-KZNFB', gender='Female', SeniorCitizen=0, Partner='No', Dependents='No', tenure=33, PhoneService='Yes', MultipleLines='Yes', InternetService='Fiber optic', OnlineSecurity='Yes', OnlineBackup='No', DeviceProtection='No', TechSupport='No', StreamingTV='No', StreamingMovies='Yes', Contract='Month-to-month', PaperlessBilling='Yes', PaymentMethod='Electronic check', MonthlyCharges=90.65, TotalCharges='2989.6', Churn='No'), Row(customerID='7113-HIPFI', gender='Male', SeniorCitizen=0, Partner='Yes', Dependents='Yes', tenure=66, PhoneService='Yes', MultipleLines='No', InternetService='DSL', OnlineSecurity='Yes', OnlineBackup='Yes', DeviceProtection='Yes', TechSupport='Yes', StreamingTV='No', StreamingMovies='No', Contract='Two year', PaperlessBilling='No', PaymentMethod='Mailed check', MonthlyCharges=65.85, TotalCharges='4097.05', Churn='No'), Row(customerID='7901-IIDQV', gender='Male', SeniorCitizen=0, Partner='No', Dependents='No', tenure=24, PhoneService='Yes', Multipl

In [27]:
# 2) Count
print(df.rdd.count())

7043


In [29]:
# 3) Save as Text File
df.rdd.saveAsTextFile(path=f"{icp4_data_path}/output")

# (see screenshots)